In [172]:
#### IMPORTS

import re
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import (permutations, combinations, chain, cycle, product, islice, 
                         takewhile, zip_longest, count as count_from)
from functools   import lru_cache
from heapq import (heappush, heappop, nlargest, nsmallest)

from pprint import pprint as p, pformat as pf
import toolz.curried as t
from tqdm import tqdm_notebook as tq

#### CONSTANTS

alphabet = 'abcdefghijklmnopqrstuvwxyz'
ALPHABET = alphabet.upper()
infinity = float('inf')

#### SIMPLE UTILITY FUNCTIONS

cat = ''.join

def ints(start, end, step=1):
    "The integers from start to end, inclusive: range(start, end+1)"
    return range(start, end + 1, step)

def first(iterable, default=None): 
    "The first item in an iterable, or default if it is empty."
    return next(iter(iterable), default)

def first_true(iterable, pred=None, default=None):
    """Returns the first true value in the iterable.
    If no true value is found, returns *default*
    If *pred* is not None, returns the first item
    for which pred(item) is true."""
    # first_true([a,b,c], default=x) --> a or b or c or x
    # first_true([a,b], fn, x) --> a if fn(a) else b if fn(b) else x
    return next(filter(pred, iterable), default)

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

def upto(iterable, maxval):
    "From a monotonically increasing iterable, generate all the values <= maxval."
    # Why <= maxval rather than < maxval? In part because that's how Ruby's upto does it.
    return takewhile(lambda x: x <= maxval, iterable)

identity = lambda x: x

def quantify(iterable, pred=bool):
    "Count how many times the predicate is true of an item in iterable."
    return sum(map(pred, iterable))

def multimap(items):
    "Given (key, val) pairs, return {key: [val, ....], ...}."
    result = defaultdict(list)
    for (key, val) in items:
        result[key].append(val)
    return result

def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

def map2d(fn, grid):
    "Apply fn to every element in a 2-dimensional grid."
    return tuple(mapt(fn, row) for row in grid)

def flatmap(fn, *args):
    "Do a map and a one-level flatten"
    return tuple(chain.from_iterable(map(fn, *args)))

def repeat(n, fn, arg, *args, **kwds):
    "Repeat arg = fn(arg) n times, return arg."
    return nth(repeatedly(fn, arg, *args, **kwds), n)

def repeatedly(fn, arg, *args, **kwds):
    "Yield arg, fn(arg), fn(fn(arg)), ..."
    yield arg
    while True:
        arg = fn(arg, *args, **kwds)
        yield arg
        
def compose(f, g): 
    "The function that computes f(g(x))."
    return lambda x: f(g(x))

#### FILE INPUT AND PARSING

def Input(day, line_parser=str.strip, test=False, file_template='data/advent2018/{}.txt'):
    "For this day's input file, return a tuple of each line parsed by `line_parser`."
    return mapt(line_parser, open(file_template.format(
        f'{day}test' if test else day
    )))

def integers(text): 
    "A tuple of all integers in a string (ignore other characters)."
    return mapt(int, re.findall(r'-?\d+', text))

In [187]:
################ 2-D points implemented using (x, y) tuples

def X(point): return point[0]
def Y(point): return point[1]

origin = (0, 0)
HEADINGS = UP, LEFT, DOWN, RIGHT = (0, -1), (-1, 0), (0, 1), (1, 0)

def turn_right(heading): return HEADINGS[HEADINGS.index(heading) - 1]
def turn_around(heading):return HEADINGS[HEADINGS.index(heading) - 2]
def turn_left(heading):  return HEADINGS[HEADINGS.index(heading) - 3]

def add(A, B): 
    "Element-wise addition of two n-dimensional vectors."
    return mapt(sum, zip(A, B))

def neighbors4(point): 
    "The four neighboring squares."
    x, y = point
    return (          (x, y-1),
            (x-1, y),           (x+1, y), 
                      (x, y+1))

def neighbors8(point): 
    "The eight neighboring squares."
    x, y = point 
    return ((x-1, y-1), (x, y-1), (x+1, y-1),
            (x-1, y),             (x+1, y),
            (x-1, y+1), (x, y+1), (x+1, y+1))

def cityblock_distance(P, Q=origin): 
    "Manhatten distance between two points."
    return sum(abs(p - q) for p, q in zip(P, Q))

def distance(P, Q=origin): 
    "Straight-line (hypotenuse) distance between two points."
    return sum((p - q) ** 2 for p, q in zip(P, Q)) ** 0.5

def king_distance(P, Q=origin):
    "Number of chess King moves between two points."
    return max(abs(p - q) for p, q in zip(P, Q))

################ Debugging 

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        arg_str = ', '.join(map(pf, args))
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, arg_str, pf(result)))
        return result
    return traced_f

def grep(pattern, iterable):
    "Print lines from iterable that match pattern."
    for line in iterable:
        if re.search(pattern, line):
            print(line)
            
class Struct:
    "A structure that can have any fields defined."
    def __init__(self, **entries): self.__dict__.update(entries)
    def __repr__(self): 
        fields = ['{}={}'.format(f, self.__dict__[f]) 
                  for f in sorted(self.__dict__)]
        return 'Struct({})'.format(', '.join(fields))

In [16]:
################ A* and Breadth-First Search (tracking states, not actions)

def always(value): return (lambda *args: value)

def Astar(start, moves_func, h_func, cost_func=always(1)):
    "Find a shortest sequence of states from start to a goal state (where h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    Path      = lambda s: ([] if (s is None) else Path(previous[s]) + [s])
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(s)
        for s2 in moves_func(s):
            g = path_cost[s] + cost_func(s, s2)
            if s2 not in path_cost or g < path_cost[s2]:
                heappush(frontier, (g + h_func(s2), s2))
                path_cost[s2] = g
                previous[s2] = s

def bfs(start, moves_func, goals):
    "Breadth-first search"
    goal_func = (goals if callable(goals) else lambda s: s in goals)
    return Astar(start, moves_func, lambda s: (0 if goal_func(s) else 1))

In [4]:
def head(iterable, n=5): return list(t.take(n, iterable))

def coords(two_d_arr):
    return [
        ((x, y), val)
        for y, line in enumerate(two_d_arr)
        for x, val in enumerate(line)
    ]

def duplicates(iterable):
    return { item for item, count in Counter(iterable).items() if count > 1 }

## Day 13

In [5]:
tracks = r'''
/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   
'''.strip('\n').split('\n')

tracks = r'''
/>-<\  
|   |  
| /<+-\
| | | v
\>+</ |
  |   ^
  \<->/
'''.strip('\n').split('\n')

tracks = Input(13, line_parser=t.identity)

In [6]:
Cart = namedtuple('Cart', 'id, pos, dir, turns')

cart_dirs = { 'v': DOWN, '^': UP, '<': LEFT, '>': RIGHT }

def junctions(tracks=tracks):
    return { pos: ch for pos, ch in coords(tracks) if ch in r'/\+' }

def carts(tracks=tracks):
    positions = [(pos, ch) for pos, ch in coords(tracks) if ch in cart_dirs]
    return deque(
        Cart(id, pos, cart_dirs[ch], cycle([turn_left, t.identity, turn_right]))
        for id, (pos, ch) in enumerate(positions)
    )

In [7]:
bends = {
    r'/': { LEFT: DOWN, DOWN: LEFT, RIGHT: UP, UP: RIGHT },
    '\\': { LEFT: UP, UP: LEFT, RIGHT: DOWN, DOWN: RIGHT },
}

def turn(cart, juncts=junctions()):
    _, pos, dir, turns = cart
    junction = juncts.get(add(pos, dir), False)
    
    if junction in bends: return bends[junction][dir]
    if junction == '+':   return next(turns)(dir)
    else:                 return dir

In [8]:
def move_cart(carts):
    if first(carts).id == min(cart.id for cart in carts):
        carts = deque(Cart(id, pos, dir, turns)
                      for id, (_, pos, dir, turns) in
                      enumerate(sorted(carts, key=lambda cart: (Y(cart.pos), X(cart.pos)))))
        
    cart = id, pos, dir, turns = carts.popleft()
    carts.append(Cart(id, add(pos, dir), turn(cart), turns))
    return carts

def collision_pos(carts):
    crashes = duplicates(cart.pos for cart in carts)
    return crashes if len(crashes) > 0 else False

In [9]:
first_true(collision_pos(c) for c in repeatedly(move_cart, carts()))

{(32, 8)}

In [10]:
def remove_collisions(carts):
    crashes = collision_pos(carts)
    return (deque(cart for cart in carts if cart.pos not in crashes)
            if crashes else carts)

In [11]:
first(c for c in repeatedly(compose(remove_collisions, move_cart), carts())
      if len(c) == 1 and first(c).id == 0)

deque([Cart(id=0, pos=(38, 38), dir=(0, 1), turns=<itertools.cycle object at 0x1093e99d8>)])

## Day 14

In [12]:
# N = 167792
N = 704321

In [13]:
Recipe = namedtuple('Recipe', 'scores, e1, e2')

def combine(recipe):
    scores, e1, e2 = recipe
    combined = scores[e1] + scores[e2]
    if combined >= 10:
        scores.append(combined // 10)
    scores.append(combined % 10)
    return Recipe(scores,
                  (e1 + scores[e1] + 1) % len(scores),
                  (e2 + scores[e2] + 1) % len(scores))

In [14]:
scores = first_true(
    tq(repeatedly(combine, Recipe([3, 7], 0, 1))),
    lambda r: len(r.scores) >= N + 10
).scores[N:]
''.join(mapt(str, scores))

'1741551073'

In [15]:
search = str(N)
scores, _, _ = first_true(
    tq(repeatedly(combine, Recipe([3, 7], 0, 1))),
    lambda r: search in cat(mapt(str, r.scores[-len(search) - 1:]))
)
scores[-len(search):]

[0, 4, 3, 2, 1, 0]

In [16]:
len(scores) - len(search) - 1

20322683

## Day 15

In [289]:
inp = Input(15, test=False)
for line in inp:
    p(line)

'################################'
'##########..####################'
'##########..G###################'
'##########..#.....########.#####'
'##########........########G#####'
'############...#..########.#####'
'################....######.#####'
'#################..G####...#####'
'################...#..#....#####'
'################...G..#.....E###'
'##############.G..........G....#'
'###########.G...G..............#'
'###########G..#####..........###'
'###########..#######.........###'
'##########.G#########........#.#'
'#########...#########....G.....#'
'#########...#########.........##'
'##..........#########.........##'
'######....G.#########.....E....#'
'##...........#######.......#...#'
'#...G.........#####E.......#####'
'##....................#..#######'
'##.G.................##.########'
'##..#GG.............###...#..###'
'#G..#..G.G........G.####.#..E###'
'#.....#.##...........###.....###'
'#######...............###EE..###'
'########.....E........###....###'
'########...........

In [290]:
def grid(cavern=inp):
    return {
        (y, x) for (x, y), val in coords(cavern)
        if val != '#'
    }

In [291]:
Unit = namedtuple('Unit', 'yx, hp, atk, typ, rds')

def order(units): return deque(sorted(units))

def units(cavern=inp, elf_atk=3):
    return [Unit((y, x), 200, elf_atk if val == 'E' else 3, val, 0)
            for (x, y), val in coords(cavern) if val in 'GE']

In [292]:
def attack(units):
    me = units.popleft()
    in_range = [u for u in units
                if u.yx in neighbors4(me.yx) and u.typ != me.typ]
    if in_range:
        hurt = min(in_range, key=lambda e: (e.hp, e.yx))
        loc = units.index(hurt)
        units[loc] = Unit(hurt.yx, hurt.hp - me.atk,
                          hurt.atk, hurt.typ, hurt.rds)
        if units[loc].hp <= 0:
            del units[loc]
            
    units.append(Unit(me.yx, me.hp, me.atk, me.typ, me.rds + 1))
    return units

In [293]:
def step(units, grid=grid()):
    adj = lambda p: set(neighbors4(p)) & grid - set(u.yx for u in units)
    
    me = units.popleft()
    enemies = [unit for unit in units if unit.typ != me.typ]
    
    if any(e.yx in neighbors4(me.yx) for e in enemies):
        units.appendleft(me)
        return units
    
    targets = sorted(flatmap(adj, (e.yx for e in enemies)))
    path = bfs(me.yx, adj, targets)
    step = nth(path, 1) if path else me.yx
    
    units.appendleft(Unit(step, me.hp, me.atk, me.typ, me.rds))
    return units

In [294]:
def move(units, grid=grid()):
    if len(set(u.rds for u in units)) == 1:
        units = order(units)

    step(units)
    return attack(units)

In [295]:
def print_map(units, grid=grid()):
    u_map = { u.yx: u for u in units }
    for y in ints(0, max(t.pluck(0, grid)) + 1):
        line = []
        units = []
        for x in ints(0, max(t.pluck(1, grid)) + 1):
            unit = u_map.get((y, x), False)
            line.append(unit.typ if unit
                        else '.' if (y, x) in grid
                        else '#')
            if unit:
                units.append(unit)
        unit_info = ', '.join(f"{u.typ}({u.hp})" for u in units)
        p(f'{cat(line)}   {unit_info}')

In [296]:
def run_n(rounds, elf_atk=3):
    return first_true(repeatedly(move, units(elf_atk=elf_atk)),
                      lambda units: all(u.rds == rounds for u in units))

print_map(run_n(47, elf_atk=15))

'################################   '
'##########..####################   '
'##########...###################   '
'##########..#.....########.#####   '
'##########........########.#####   '
'############...#..########.#####   '
'################....######.#####   '
'#################...####...#####   '
'################...#..#....#####   '
'################......#.E....###   E(95)'
'##############........E.E......#   E(77), E(197)'
'###########............E.......#   E(179)'
'###########...#####..........###   '
'###########..#######.........###   '
'##########..#########........#.#   '
'#########...#########..........#   '
'#########...#########.........##   '
'##..........#########.........##   '
'######......#########..........#   '
'##...........#######.......#...#   '
'#...........E.#####.E......#####   E(104), E(53)'
'##.................E..#..#######   E(161)'
'##..................E##.########   E(176)'
'##..#...............###...#..###   '
'#...#...............####.#...###   '
'

In [297]:
def end(units):
    return first_true(repeatedly(move, units),
                      lambda units: len(set(u.typ for u in units)) == 1)
end(units())

deque([Unit(yx=(18, 22), hp=200, atk=3, typ='G', rds=59),
       Unit(yx=(18, 24), hp=104, atk=3, typ='G', rds=59),
       Unit(yx=(18, 25), hp=200, atk=3, typ='G', rds=59),
       Unit(yx=(19, 20), hp=200, atk=3, typ='G', rds=59),
       Unit(yx=(22, 10), hp=200, atk=3, typ='G', rds=59),
       Unit(yx=(24, 12), hp=32, atk=3, typ='G', rds=59),
       Unit(yx=(25, 11), hp=200, atk=3, typ='G', rds=59),
       Unit(yx=(25, 13), hp=200, atk=3, typ='G', rds=59),
       Unit(yx=(26, 10), hp=152, atk=3, typ='G', rds=59),
       Unit(yx=(26, 12), hp=200, atk=3, typ='G', rds=59),
       Unit(yx=(27, 11), hp=200, atk=3, typ='G', rds=59),
       Unit(yx=(11, 23), hp=116, atk=3, typ='G', rds=60),
       Unit(yx=(15, 26), hp=74, atk=3, typ='G', rds=60),
       Unit(yx=(16, 22), hp=143, atk=3, typ='G', rds=60),
       Unit(yx=(16, 24), hp=170, atk=3, typ='G', rds=60),
       Unit(yx=(16, 27), hp=200, atk=3, typ='G', rds=60),
       Unit(yx=(17, 21), hp=200, atk=3, typ='G', rds=60),
       Unit(yx=(

In [299]:
ending = end(units())
min(u.rds for u in ending) * sum(u.hp for u in ending)

178003

In [300]:
is_elf = lambda u: u.typ == 'E'
n_elves = quantify(units(), is_elf)
min_atk = first_true(
    count_from(4),
    lambda atk: quantify(end(units(elf_atk=atk)), is_elf) == n_elves
)
min_atk

23

In [301]:
ending = end(units(elf_atk=23))
min(u.rds for u in ending) * sum(u.hp for u in ending)

48722